In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


TEMPORARY

In [ ]:
import pandas as pd
hsol=pd.read_csv('/content/drive/MyDrive/Workspace/NLE/DATASETS/HSOL/hsol.csv')

In [ ]:
hsol=hsol[['tweet','class']]
hsol.columns = ["tweet", "label"]
hsol.label =hsol.label.replace({0: 1,2:0})
hsol['label'].value_counts() 

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


1    20620
0     4163
Name: label, dtype: int64

In [ ]:
olid=pd.read_csv('/content/drive/MyDrive/Workspace/NLE/DATASETS/OLID/olid-training-v1.0.tsv', sep='\t', header=0)
olid=olid[['tweet','subtask_a']]
olid.columns = ["tweet", "label"]
olid.label =olid.label.replace({"OFF": 1,"NOT":0})
olid['label'].value_counts() #

0    8840
1    4400
Name: label, dtype: int64

In [ ]:
olid2=pd.read_csv('/content/drive/MyDrive/Workspace/NLE/DATASETS/OLID/testset-levela.tsv', sep='\t', header=0)
o2=pd.read_csv('/content/drive/MyDrive/Workspace/NLE/DATASETS/OLID/labels-levela.csv',header=None)
new=pd.concat([olid2,o2],axis=1)
new=new[['tweet',1]]
new.columns=['tweet','label']
new.label=new.label.replace({"OFF":1,"NOT":0})
new['label'].value_counts() #

0    620
1    240
Name: label, dtype: int64

In [ ]:
ethos=pd.read_csv('/content/drive/MyDrive/Workspace/NLE/DATASETS/ethos/ethos_binary.csv')
ethos.columns = ["tweet", "label"] #1:hate 0:no hate
ethos['label'].value_counts()

0    565
1    433
Name: label, dtype: int64

hasoc

In [ ]:
hasoc2019=pd.read_csv('/content/drive/MyDrive/Workspace/NLE/hasoc2k19/english_dataset.tsv', sep='\t', header=0)
hasoc2019=hasoc2019[['text','task_1']]
hasoc2019.columns=['tweet','label']
hasoc2019.label=hasoc2019.label.replace({"NOT":0,"HOF":1})
hasoc2019['label'].value_counts()

0    3591
1    2261
Name: label, dtype: int64

In [ ]:
hasoc2021=pd.read_csv('/content/drive/MyDrive/Workspace/hasoc/hasoc2021/train.csv')
hasoc2021=hasoc2021[['text','task_1']]
hasoc2021.columns=['tweet','label']
hasoc2021.label=hasoc2021.label.replace({"NOT":0,"HOF":1})
hasoc2021['label'].value_counts()

1    2501
0    1342
Name: label, dtype: int64

special dset

In [ ]:
train1=pd.concat([hsol,olid,new,ethos,hasoc2019,hasoc2021],axis=0,ignore_index=True) #withall

In [ ]:
train1

,tweet,label
0,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1
...,...,...
49571,@BBCNews Let the dog deal with the wanker once...,1
49572,India has suffered a lot. That Chinese bastard...,1
49573,People didn't give 300+ seats majority to BJP ...,1
49574,"@KanganaTeam This is such a vile, xenophobic a...",1


In [ ]:
train2=pd.concat([hasoc2019,hasoc2021],axis=0,ignore_index=True) #withjusthasoc

In [ ]:
train2

,tweet,label
0,#DhoniKeepsTheGlove | WATCH: Sports Minister K...,0
1,@politico No. We should remember very clearly ...,1
2,@cricketworldcup Guess who would be the winner...,0
3,Corbyn is too politically intellectual for #Bo...,0
4,All the best to #TeamIndia for another swimmin...,0
...,...,...
9690,@BBCNews Let the dog deal with the wanker once...,1
9691,India has suffered a lot. That Chinese bastard...,1
9692,People didn't give 300+ seats majority to BJP ...,1
9693,"@KanganaTeam This is such a vile, xenophobic a...",1


In [ ]:
!pip install wordsegment
from collections import OrderedDict
import re
from wordsegment import load, segment
import numpy as np
load()
def hash_preprocessing(s):
    s=re.findall(r'\B#\w*[a-zA-Z]+\w*', s)
    return s

def hash_fix(h):
    h1 = re.sub(r'[0-9]+', '', h)
    h2 = re.sub(r'#', '', h1)
    h3 = segment(str(h2))
    h4 = ' '.join(map(str, h3)) 
    return h4

def hash_dict(df,hash_col):
    # Create a datafame of all hashtags in a column and their counts
    # Note: hashtags are in lists inside a cell e.g. [#hash1, #hash2] 
    tag_counts = df[hash_col].apply(pd.Series).stack().value_counts().to_frame()
    tag_counts = tag_counts.reset_index()
    tag_counts.columns = ['hash','freq']
    # Remove numbers and segment multiple words using hash fix
    tag_counts = tag_counts.assign(clean_tag = tag_counts.hash.apply(lambda x: hash_fix(x)))
    # Create a dictionary of the hashtags and their clean strings
    tag_counts.set_index('hash', inplace=True)
    tag_dict = tag_counts['clean_tag'].to_dict()
    return tag_dict

def featureExtractor(df):
    # b = df['tweet'].astype(str)
    # # c = df['abstract_text'].astype(str)
    # df['tweet']=b
    f_df=df[['tweet','label']]
    # L=len(f_df)
    # l1=int(L/4)
    # l2=int(L/2)
    # l3=int(3*L/2)
    # df1,df2,df3,df4=f_df[:l1],f_df[l1:l2],f_df[l2:l3],f_df[l3:]
    # df1['features'] = (df1['features'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))
    # df2['features'] = (df2['features'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))
    # df3['features'] = (df3['features'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))
    # df4['features'] = (df4['features'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))
    final=f_df
    # final = pd.concat([df1, df2,df3,df4], ignore_index=True)
    final['tweet']=final['tweet']. str.lower()
    hash_preprocessed = np.array([hash_preprocessing(text) for text in final['tweet']])
    hashtags=pd.DataFrame(hash_preprocessed)
    tag_dict = hash_dict(hashtags,0)
    final['tweet']=final['tweet'].replace(tag_dict, regex=True)
    final['tweet']=final['tweet'].str.replace("https\S+", " ")
    final['tweet']=final['tweet'].str.replace("rt @[A-Za-z0-9_]+","")
    final['tweet']=final['tweet'].str.replace("@[A-Za-z0-9_]+","")
    final['tweet']=final['tweet'].str.replace("[^a-zA-Z0-9,.'*’?]", " ")
    final['tweet']=final['tweet'].str.replace('url', "")
    final['tweet']=final['tweet'].str.replace('\n', "")
    # final['tweet'] =final['tweet'] .apply(lambda x: re.sub(r'\b\w{1,1}\b', '', x))
      
    # final['tweet']= (final['tweet'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))
    final['tweet']=final['tweet'].apply(lambda x: " ".join(x.split()))
    

    return final

     |████████████████████████████████| 4.8 MB 10.8 MB/s 


In [ ]:
d1=featureExtractor(train1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
d2=featureExtractor(train2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [ ]:
d1.to_csv('/content/drive/MyDrive/Workspace/NLE/DATASETS/special/alltrain.csv',index=False)
d2.to_csv('/content/drive/MyDrive/Workspace/NLE/DATASETS/special/onlyhasoctrain.csv',index=False)

In [ ]:
d1['label'].value_counts()

1    30455
0    19121
Name: label, dtype: int64

In [ ]:
d2['label'].value_counts()

0    4933
1    4762
Name: label, dtype: int64